# Stokes Equation with Adjoint Gradients

In this demo we consider the Stokes equation:

$$
\begin{align}
- \nu \Delta \mathbf{u} + \nabla p &= \mathbf{f} \quad \text{in } \Omega \\
\nabla \cdot \mathbf{u} &= 0 \quad \text{in } \Omega
\end{align}
$$

with Dirichlet boundary conditions:

$$
\begin{align}
\mathbf{u} &= \mathbf{g} \quad \text{on circle boundary} \\
\mathbf{u} &= \mathbf{h} \quad \text{on inlet boundary} \\
\mathbf{u} &= 0 \quad \text{on wall boundaries} \\
p &= 0 \quad \text{on outlet boundary}
\end{align}
$$

where:
- $\mathbf{u} : \Omega \to \mathbb{R}^2$ is the unknown velocity field
- $p : \Omega \to \mathbb{R}$ is the unknown pressure field
- $\nu : \Omega \to \mathbb{R}$ is the viscosity field
- $\mathbf{f} : \Omega \to \mathbb{R}^2$ is the body force field
- $\mathbf{h} : \Omega \to \mathbb{R}^2$ is the inlet velocity boundary condition
- $\mathbf{g} : \Omega \to \mathbb{R}^2$ is the circle velocity boundary condition

The domain $\Omega$ is defined as a rectangle with a circle cut out of it, similar to the DFG 2D-3 benchmark of flow past a cylinder. The inlet boundary is the left boundary, the outlet boundary is the right boundary, and the wall boundaries are the top and bottom boundaries.

We define the objective function (or quantity of interest):

$$
J(\mathbf{u}) = \frac{1}{2}\int_\Omega \nabla\mathbf{u} : \nabla\mathbf{u} \, d\mathbf{x} + \frac{\alpha}{2}\int_{\partial\Omega_{\text{Circle}}} \mathbf{g} \cdot \mathbf{g} \, ds
$$

with the regularization parameter $\alpha = 10$.

The weak formulation of the problem reads: find $(\mathbf{u}, p) \in V \times Q$ such that:

$$
a((\mathbf{u}, p), (\mathbf{v}, q)) = L((\mathbf{v}, q)) \quad \forall (\mathbf{v}, q) \in V \times Q
$$

where:

$$
\begin{align}
a((\mathbf{u}, p), (\mathbf{v}, q)) &= \int_\Omega \nu \nabla \mathbf{u} : \nabla \mathbf{v} \, d\mathbf{x} - \int_\Omega (\nabla \cdot \mathbf{v}) p \, d\mathbf{x} - \int_\Omega (\nabla \cdot \mathbf{u}) q \, d\mathbf{x} \\
L(\mathbf{v}, q) &= \int_\Omega \mathbf{f} \cdot \mathbf{v} \, d\mathbf{x}
\end{align}
$$

We solve this problem with a residual equation:

$$
F((\mathbf{u}, p)) = a((\mathbf{u}, p), (\mathbf{v}, q)) - L((\mathbf{v}, q)) = 0
$$

In [ ]:
import numpy as np
import ufl
from dolfinx import fem, io, nls
from mpi4py import MPI
from petsc4py.PETSc import ScalarType

from dolfinx_adjoint import *

import gmsh

We first need to create a graph object to store the computational graph. This is done explicitly to maintain the guideline of FEniCSx. Every function that is created with a graph object will be added to the graph and its gradient will be computed automatically.

In [ ]:
graph_ = Graph()

In [ ]:
# Mesh parameters
gmsh.initialize()
L = 2.2
H = 0.41
c_x = 0.2
c_y = 0.2
r = 0.05
gdim = 2
mesh_comm = MPI.COMM_WORLD
model_rank = 0

fluid_marker = 1
inlet_marker, outlet_marker, wall_marker, obstacle_marker = 2, 3, 4, 5
inflow, outflow, walls, obstacle = [], [], [], []

# Mesh generation
if mesh_comm.rank == model_rank:
    rectangle = gmsh.model.occ.addRectangle(0, 0, 0, L, H, tag=1)
    circle = gmsh.model.occ.addDisk(c_x, c_y, 0, r, r)
    fluid = gmsh.model.occ.cut([(gdim, rectangle)], [(gdim, circle)])
    gmsh.model.occ.synchronize()

    volumes = gmsh.model.getEntities(dim=gdim)
    assert len(volumes) == 1
    gmsh.model.addPhysicalGroup(volumes[0][0], [volumes[0][1]], fluid_marker)
    gmsh.model.setPhysicalName(volumes[0][0], fluid_marker, "Fluid")

    boundaries = gmsh.model.getBoundary(volumes, oriented=False)
    for boundary in boundaries:
        center_of_mass = gmsh.model.occ.getCenterOfMass(boundary[0], boundary[1])
        if np.allclose(center_of_mass, [0, H / 2, 0]):
            inflow.append(boundary[1])
        elif np.allclose(center_of_mass, [L, H / 2, 0]):
            outflow.append(boundary[1])
        elif np.allclose(center_of_mass, [L / 2, H, 0]) or np.allclose(
            center_of_mass, [L / 2, 0, 0]
        ):
            walls.append(boundary[1])
        else:
            obstacle.append(boundary[1])
    gmsh.model.addPhysicalGroup(1, walls, wall_marker)
    gmsh.model.setPhysicalName(1, wall_marker, "Walls")
    gmsh.model.addPhysicalGroup(1, inflow, inlet_marker)
    gmsh.model.setPhysicalName(1, inlet_marker, "Inlet")
    gmsh.model.addPhysicalGroup(1, outflow, outlet_marker)
    gmsh.model.setPhysicalName(1, outlet_marker, "Outlet")
    gmsh.model.addPhysicalGroup(1, obstacle, obstacle_marker)
    gmsh.model.setPhysicalName(1, obstacle_marker, "Obstacle")

    gmsh.model.mesh.field.add("Distance", 1)
    gmsh.model.mesh.field.setNumbers(1, "EdgesList", obstacle)
    gmsh.model.mesh.field.add("Threshold", 2)
    gmsh.model.mesh.field.setNumber(2, "IField", 1)
    gmsh.model.mesh.field.setNumber(2, "LcMin", 0.01)
    gmsh.model.mesh.field.setNumber(2, "LcMax", 0.04)
    gmsh.model.mesh.field.setNumber(2, "DistMin", 0)
    gmsh.model.mesh.field.setNumber(2, "DistMax", H)
    gmsh.model.mesh.field.add("Min", 5)
    gmsh.model.mesh.field.setNumbers(5, "FieldsList", [2])
    gmsh.model.mesh.field.setAsBackgroundMesh(5)
    gmsh.model.mesh.generate(2)

mesh, _, ft = io.gmshio.model_to_mesh(gmsh.model, mesh_comm, model_rank, gdim=gdim)
ft.name = "Facet markers"

In [ ]:
u_elem = ufl.VectorElement("CG", mesh.ufl_cell(), 2)
p_elem = ufl.FiniteElement("CG", mesh.ufl_cell(), 1)

v_elem = ufl.MixedElement([u_elem, p_elem])
V = fem.functionspace(mesh, v_elem)
V_u, V_u_map = V.sub(0).collapse()
V_p, V_p_map = V.sub(1).collapse()

up = fem.Function(V, name="up", graph=graph_)
u, p = ufl.split(up)

vq = ufl.TestFunction(V)
v, q = ufl.split(vq)

In [ ]:
# Boundary conditions
h = fem.Function(V_u, name="f")  # Inflow Dirichlet boundary condition
speed = 0.3
h.interpolate(lambda x: np.stack((speed * 4 * x[1] * (H - x[1]) / (H * H), 0.0 * x[0])))
g = fem.Function(V_u, name="g", graph=graph_)  # Circle Dirichlet boundary condition
noslip = fem.Function(V_u, name="noslip")  # No-slip boundary condition
outflow = fem.Function(V_p, name="outflow")  # Outflow pressure boundary condition
outflow.interpolate(lambda x: 0.0 * x[0] + 0.0)

dofs_walls = fem.locate_dofs_topological(
    (V.sub(0), V_u), 1, ft.indices[ft.values == wall_marker]
)
dofs_inflow = fem.locate_dofs_topological(
    (V.sub(0), V_u), 1, ft.indices[ft.values == inlet_marker]
)
dofs_outflow = fem.locate_dofs_topological(
    (V.sub(1), V_p), 1, ft.indices[ft.values == outlet_marker]
)
dofs_obstacle = fem.locate_dofs_topological(
    (V.sub(0), V_u), 1, ft.indices[ft.values == obstacle_marker]
)

bcs = [
    fem.dirichletbc(h, dofs_inflow, V.sub(0)),
    fem.dirichletbc(g, dofs_obstacle, V.sub(0), graph=graph_, map=V_u_map),
    fem.dirichletbc(noslip, dofs_walls, V.sub(0)),
    fem.dirichletbc(outflow, dofs_outflow, V.sub(1)),
]

# Parameters
nu = fem.Constant(mesh, ScalarType(1.0), name="ν", graph=graph_)
alpha = 10.0
f = fem.Function(V_u, name="f")
f.interpolate(lambda x: (0.0 * x[0], 0.0 + 0.0 * x[1]))

In [ ]:
a = (
    nu * ufl.inner(ufl.grad(u), ufl.grad(v)) * ufl.dx
    - ufl.div(v) * p * ufl.dx
    + q * ufl.div(u) * ufl.dx
)
L = ufl.inner(f, v) * ufl.dx
F = a - L

In [ ]:
problem = fem.petsc.NonlinearProblem(F, up, bcs=bcs, graph=graph_)
solver = nls.petsc.NewtonSolver(MPI.COMM_WORLD, problem, graph=graph_)

solver.solve(up, graph=graph_)

if MPI.COMM_WORLD.rank == 0:
    print("Forward problem solved successfully!")

In [ ]:
dObs = ufl.Measure("ds", domain=mesh, subdomain_data=ft, subdomain_id=obstacle_marker)
J_form = (
    0.5 * ufl.inner(ufl.grad(u), ufl.grad(u)) * ufl.dx
    + alpha / 2 * ufl.inner(g, g) * dObs
)

J = fem.assemble_scalar(fem.form(J_form, graph=graph_), graph=graph_)

if MPI.COMM_WORLD.rank == 0:
    print(f"Objective function value: J(u, p) = {J}")

Compute adjoint gradients with respect to the viscosity and boundary condition:

In [ ]:
dJdnu = graph_.backprop(id(J), id(nu))
dJdg = graph_.backprop(id(J), id(g))

if MPI.COMM_WORLD.rank == 0:
    print("\n" + "="*60)
    print("Gradient Results:")
    print("="*60)
    print(f"J(u, p) = {J}")
    print(f"dJ/dν = {dJdnu}")
    print(f"||dJ/dg||_L2 = {np.sqrt(np.dot(dJdg, dJdg))}")
    print("="*60)

gmsh.finalize()